Code used to download daily stock price using python package yfinance. 
A file was created as a csv to store downloaded price on the initial run.
After initial run, only new data are downloaded, then append to the existing data. 

In [1]:
# Import require libraries

!pip install yfinance
!pip install yfinance --upgrade --no-cache-dir
!pip install pandas-datareader
!pip install beautifulsoup4

import yfinance as yf
import yahooquery as yq
import numpy as np
import datetime as dt
import bs4 as bs
from datetime import datetime
import csv

import requests
import pandas as pd
from pandas import DataFrame
import pandas_datareader.data as web
import pandas_datareader as web
from concurrent import futures

Requirement already up-to-date: yfinance in /Applications/anaconda3/lib/python3.8/site-packages (0.1.68)


In [2]:
# Get tickers from csv file

reader1 = pd.read_csv('/Users/songyang/Desktop/XUE/Data analysis projects/3DPrintingStockMarket/3DPrintingCompaniesTickers.csv')
tickers1 = reader1['ticker'].tolist()
print(tickers1)

['688333.SS', 'DDD', 'DM', 'SSYS', 'VJET', 'NNDM', 'ONVO', 'SHPW', 'MKFG', 'AM3D.DE', 'PRLB', 'MTLS', 'VLD', 'MSVT.TA', 'AAPL', 'TSLA', '1211.HK', 'TCEHY', 'GE', 'SIE.DE']


In [3]:
# get marketCap and other basic informations of companies

market_cap_data = web.get_quote_yahoo(tickers1)[['currency','financialCurrency', 'fullExchangeName', 'longName', 'shortName', 
                                                 'exchangeTimezoneName', 'marketCap', 'price']]
marketCapdf = pd.DataFrame(zip(tickers1, market_cap_data.currency, market_cap_data.financialCurrency, market_cap_data.fullExchangeName,
                              market_cap_data.longName, market_cap_data.shortName, market_cap_data.exchangeTimezoneName,
                              market_cap_data.marketCap, market_cap_data.price)) 
marketCapdf.columns = ['ticker','currency','financialCurrency', 'fullExchangeName', 'longName', 'shortName', 
                       'exchangeTimezoneName', 'marketCap', 'price']
pd.set_option('display.max_rows', 500)

marketCapdf['currency'] = marketCapdf['currency'].str.upper()
marketCapdf.to_csv('/Users/songyang/Desktop/XUE/Data analysis projects/3DPrintingStockMarket/3DPrintingMarketCap.csv', index=False)

"""
# More keys that can be used
      ['language', 'region', 'quoteType', 'triggerable', 'quoteSourceName',
       'currency', 'preMarketChange', 'preMarketChangePercent',
       'preMarketTime', 'preMarketPrice', 'regularMarketChange',
       'regularMarketChangePercent', 'regularMarketTime', 'regularMarketPrice',
       'regularMarketDayHigh', 'regularMarketDayRange', 'regularMarketDayLow',
       'regularMarketVolume', 'regularMarketPreviousClose', 'bid', 'ask',
       'bidSize', 'askSize', 'fullExchangeName', 'financialCurrency',
       'regularMarketOpen', 'averageDailyVolume3Month',
       'averageDailyVolume10Day', 'fiftyTwoWeekLowChange',
       'fiftyTwoWeekLowChangePercent', 'fiftyTwoWeekRange',
       'fiftyTwoWeekHighChange', 'fiftyTwoWeekHighChangePercent',
       'fiftyTwoWeekLow', 'fiftyTwoWeekHigh', 'dividendDate',
       'earningsTimestamp', 'earningsTimestampStart', 'earningsTimestampEnd',
       'trailingAnnualDividendRate', 'trailingPE',
       'trailingAnnualDividendYield', 'marketState', 'epsTrailingTwelveMonths',
       'epsForward', 'sharesOutstanding', 'bookValue', 'fiftyDayAverage',
       'fiftyDayAverageChange', 'fiftyDayAverageChangePercent',
       'twoHundredDayAverage', 'twoHundredDayAverageChange',
       'twoHundredDayAverageChangePercent', 'marketCap', 'forwardPE',
       'priceToBook', 'sourceInterval', 'exchangeDataDelayedBy', 'tradeable',
       'firstTradeDateMilliseconds', 'priceHint', 'exchange', 'shortName',
       'longName', 'messageBoardId', 'exchangeTimezoneName',
       'exchangeTimezoneShortName', 'gmtOffSetMilliseconds', 'market',
       'esgPopulated', 'price']
"""

"\n# More keys that can be used\n      ['language', 'region', 'quoteType', 'triggerable', 'quoteSourceName',\n       'currency', 'preMarketChange', 'preMarketChangePercent',\n       'preMarketTime', 'preMarketPrice', 'regularMarketChange',\n       'regularMarketChangePercent', 'regularMarketTime', 'regularMarketPrice',\n       'regularMarketDayHigh', 'regularMarketDayRange', 'regularMarketDayLow',\n       'regularMarketVolume', 'regularMarketPreviousClose', 'bid', 'ask',\n       'bidSize', 'askSize', 'fullExchangeName', 'financialCurrency',\n       'regularMarketOpen', 'averageDailyVolume3Month',\n       'averageDailyVolume10Day', 'fiftyTwoWeekLowChange',\n       'fiftyTwoWeekLowChangePercent', 'fiftyTwoWeekRange',\n       'fiftyTwoWeekHighChange', 'fiftyTwoWeekHighChangePercent',\n       'fiftyTwoWeekLow', 'fiftyTwoWeekHigh', 'dividendDate',\n       'earningsTimestamp', 'earningsTimestampStart', 'earningsTimestampEnd',\n       'trailingAnnualDividendRate', 'trailingPE',\n       'trail

In [4]:
# Download historical stock data

reader = csv.reader(open('/Users/songyang/Desktop/XUE/Data analysis projects/3DPrintingStockMarket/3DPrintingMarketCap.csv', 'r'))
#df = pd.read_csv('/Users/songyang/Desktop/XUE/Data analysis projects/StockMarketDataAnalysis_Tableau-master/SP500CompanyInfo.csv',delimiter=',', encoding='utf-8', skiprows=[0], header=1)
next(reader)
tickers = {}
for column in reader:
    k = column[5]
    v = column[0]
    tickers[k] = v
print(tickers)
start = dt.datetime(2018,1,1)
end = dt.datetime.now()
df_stocks = pd.DataFrame()
for key, value in tickers.items():
    download = yf.download(value, start, end)
    download = pd.DataFrame(download)
    download['Ticker'] = value
    df_stocks = df_stocks.append(download)
df_stocks.to_csv('/Users/songyang/Desktop/XUE/Data analysis projects/3DPrintingStockMarket/get3DPrintingStockData.csv')

{"XI'AN BRIGHT LASER TECHNOLOGIES": '688333.SS', '3D Systems Corporation': 'DDD', 'Desktop Metal, Inc.': 'DM', 'Stratasys, Ltd.': 'SSYS', 'voxeljet AG': 'VJET', 'Nano Dimension Ltd.': 'NNDM', 'Organovo Holdings, Inc.': 'ONVO', 'Shapeways Holdings, Inc.': 'SHPW', 'Markforged Holding Corporation': 'MKFG', 'SLM SOLUTIONS GRP AG': 'AM3D.DE', 'Proto Labs, Inc.': 'PRLB', 'Materialise NV': 'MTLS', 'Velo3D, Inc.': 'VLD', 'MASSIVIT': 'MSVT.TA', 'Apple Inc.': 'AAPL', 'Tesla, Inc.': 'TSLA', 'BYD COMPANY': '1211.HK', 'TENCENT HOLDINGS LIMITED': 'TCEHY', 'General Electric Company': 'GE', 'SIEMENS AG': 'SIE.DE'}
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************

In [5]:
# import currency converter

!pip install CurrencyConverter
from currency_converter import CurrencyConverter
c = CurrencyConverter()
#c.convert(1, 'EUR', 'CNY')

In [6]:
# Convert currency to USD

pd.options.mode.chained_assignment = None  # default='warn'

reader1 = pd.read_csv('/Users/songyang/Desktop/XUE/Data analysis projects/3DPrintingStockMarket/3DPrintingMarketCap.csv')
reader1['currencyRatesToUSD'] = ""
for i in range (len(reader1.currency)):
    if reader1.currency[i] != 'TWD' and reader1.currency[i] != 'ILA':
        reader1['currencyRatesToUSD'][i] = c.convert(1, 'USD', reader1.currency[i])
    if reader1.currency[i] == 'TWD':
        reader1['currencyRatesToUSD'][i] = 27.62 #2022/01/03
    if reader1.currency[i] == 'ILA':
        reader1['currencyRatesToUSD'][i] = 308.582 #2022/01/03
        
reader1['marketCap_USD'] = ""
reader1['price_USD'] = ""

for j in range (len(reader1.currency)):
    reader1['marketCap_USD'][j] = reader1.marketCap[j] /reader1.currencyRatesToUSD[j]
    reader1['price_USD'][j] = reader1.price[j] /reader1.currencyRatesToUSD[j]

In [7]:
# Add empty column of other desired company information for analysis
reader1['fullTimeEmployees'] = ""
#reader1['longBusinessSummary'] = ""
reader1['city'] = ""
reader1['country'] = ""
reader1['totalRevenue2021*'] = ""
reader1['totalRevenue2020'] = ""
reader1['totalRevenue2019'] = ""
reader1['totalRevenue2018'] = ""
reader1['totalRevenue2017'] = ""
reader1['netIncome2021*'] = ""
reader1['netIncome2020'] = ""
reader1['netIncome2019'] = ""
reader1['netIncome2018'] = ""
reader1['netIncome2017'] = ""
reader1['researchDevelopment2021*'] = ""
reader1['researchDevelopment2020'] = ""
reader1['researchDevelopment2019'] = ""
reader1['researchDevelopment2018'] = ""
reader1['researchDevelopment2017'] = ""
reader1['sharesOutstanding'] = ""
pd.set_option('display.max_columns', 500)

In [8]:
# Fill columns of 'fullTimeEmployees', 'city', 'country' 
CompanyInfo = {}
for i in range (len(reader1.ticker)):
    CompanyInfo[i] = yf.Ticker(reader1.ticker[i]).info
    if 'fullTimeEmployees' in CompanyInfo[i].keys():
        reader1['fullTimeEmployees'][i] = CompanyInfo[i]['fullTimeEmployees']
    if 'city' in CompanyInfo[i].keys():
        reader1['city'][i] = CompanyInfo[i]['city']
    if 'country' in CompanyInfo[i].keys():
        reader1['country'][i] = CompanyInfo[i]['country']
    if 'sharesOutstanding' in CompanyInfo[i].keys():
        reader1['sharesOutstanding'][i] = CompanyInfo[i]['sharesOutstanding']

In [9]:
# Fill columns of 'totalRevenue', 'NetIncome', 'ResearchDevelopment'.

CompanyFinancials2021 = {}
CompanyFinancials2017_2020 = {}

for i in range (len(reader1.ticker)):
    
    CompanyFinancials2021[i] = pd.DataFrame()
    CompanyFinancials2021[i] = CompanyFinancials2021[i].append(yf.Ticker(reader1.ticker[i]).quarterly_financials)
    CompanyFinancials2017_2020[i] = pd.DataFrame()
    CompanyFinancials2017_2020[i] = CompanyFinancials2017_2020[i].append(yf.Ticker(reader1.ticker[i]).financials)
# Total Revenue of 2021     
    if 'Total Revenue' in CompanyFinancials2021[i].index:
        if (pd.to_datetime('2021-09-30') in CompanyFinancials2021[i].keys() and 
            pd.to_datetime('2021-06-30') in CompanyFinancials2021[i].keys() and
            pd.to_datetime('2021-03-31') in CompanyFinancials2021[i].keys() and
            pd.to_datetime('2020-12-31') in CompanyFinancials2021[i].keys() and
            type(CompanyFinancials2021[i].loc['Total Revenue','2021-09-30'].values[0]) == float and
            type(CompanyFinancials2021[i].loc['Total Revenue','2021-06-30'].values[0]) == float and
            type(CompanyFinancials2021[i].loc['Total Revenue','2021-03-31'].values[0]) == float and
            type(CompanyFinancials2021[i].loc['Total Revenue','2020-12-31'].values[0]) == float
           ):
            reader1['totalRevenue2021*'][i] = (CompanyFinancials2021[i].loc['Total Revenue','2021-09-30'].values[0] +
                                               CompanyFinancials2021[i].loc['Total Revenue','2021-06-30'].values[0] +
                                               CompanyFinancials2021[i].loc['Total Revenue','2021-03-31'].values[0] +
                                               CompanyFinancials2021[i].loc['Total Revenue','2020-12-31'].values[0] ) 
# Total Revenue of 2017-2020 
    if 'Total Revenue' in CompanyFinancials2017_2020[i].index:
        
        if pd.to_datetime('2020-12-31') in CompanyFinancials2017_2020[i].keys():
            reader1['totalRevenue2020'][i] = CompanyFinancials2017_2020[i].loc['Total Revenue','2020-12-31'].values[0]  
        if pd.to_datetime('2019-12-31') in CompanyFinancials2017_2020[i].keys():
            reader1['totalRevenue2019'][i] = CompanyFinancials2017_2020[i].loc['Total Revenue','2019-12-31'].values[0]   
        if pd.to_datetime('2018-12-31') in CompanyFinancials2017_2020[i].keys():
            reader1['totalRevenue2018'][i] = CompanyFinancials2017_2020[i].loc['Total Revenue','2018-12-31'].values[0]
        if pd.to_datetime('2017-12-31') in CompanyFinancials2017_2020[i].keys():
            reader1['totalRevenue2017'][i] = CompanyFinancials2017_2020[i].loc['Total Revenue','2017-12-31'].values[0]

# Net Income of 2021     
    if 'Net Income' in CompanyFinancials2021[i].index:
        if (pd.to_datetime('2021-09-30') in CompanyFinancials2021[i].keys() and 
            pd.to_datetime('2021-06-30') in CompanyFinancials2021[i].keys() and
            pd.to_datetime('2021-03-31') in CompanyFinancials2021[i].keys() and
            pd.to_datetime('2020-12-31') in CompanyFinancials2021[i].keys() and
            type(CompanyFinancials2021[i].loc['Net Income','2021-09-30'].values[0]) == float and
            type(CompanyFinancials2021[i].loc['Net Income','2021-06-30'].values[0]) == float and
            type(CompanyFinancials2021[i].loc['Net Income','2021-03-31'].values[0]) == float and
            type(CompanyFinancials2021[i].loc['Net Income','2020-12-31'].values[0]) == float
           ):
            reader1['netIncome2021*'][i] = (CompanyFinancials2021[i].loc['Net Income','2021-09-30'].values[0] +
                                            CompanyFinancials2021[i].loc['Net Income','2021-06-30'].values[0] +
                                            CompanyFinancials2021[i].loc['Net Income','2021-03-31'].values[0] +
                                            CompanyFinancials2021[i].loc['Net Income','2020-12-31'].values[0] ) 
        else:
            reader1['netIncome2021*'][i] = 0        
            
# Net Income of 2017-2020            
    if 'Net Income' in CompanyFinancials2017_2020[i].index:
        if pd.to_datetime('2020-12-31') in CompanyFinancials2017_2020[i].keys():
            reader1['netIncome2020'][i] = CompanyFinancials2017_2020[i].loc['Total Revenue','2020-12-31'].values[0]
        if pd.to_datetime('2019-12-31') in CompanyFinancials2017_2020[i].keys():
            reader1['netIncome2019'][i] = CompanyFinancials2017_2020[i].loc['Total Revenue','2019-12-31'].values[0]
        if pd.to_datetime('2018-12-31') in CompanyFinancials2017_2020[i].keys():
            reader1['netIncome2018'][i] = CompanyFinancials2017_2020[i].loc['Total Revenue','2018-12-31'].values[0]
        if pd.to_datetime('2017-12-31') in CompanyFinancials2017_2020[i].keys():
            reader1['netIncome2017'][i] = CompanyFinancials2017_2020[i].loc['Total Revenue','2017-12-31'].values[0]

# Research Development of 2021     
    if 'Research Development' in CompanyFinancials2021[i].index:
        if (pd.to_datetime('2021-09-30') in CompanyFinancials2021[i].keys() and 
            pd.to_datetime('2021-06-30') in CompanyFinancials2021[i].keys() and
            pd.to_datetime('2021-03-31') in CompanyFinancials2021[i].keys() and
            pd.to_datetime('2020-12-31') in CompanyFinancials2021[i].keys() and
            type(CompanyFinancials2021[i].loc['Research Development','2021-09-30'].values[0]) == float and
            type(CompanyFinancials2021[i].loc['Research Development','2021-06-30'].values[0]) == float and
            type(CompanyFinancials2021[i].loc['Research Development','2021-03-31'].values[0]) == float and
            type(CompanyFinancials2021[i].loc['Research Development','2020-12-31'].values[0]) == float
           ):
            reader1['researchDevelopment2021*'][i] = (CompanyFinancials2021[i].loc['Research Development','2021-09-30'].values[0] +
                                                      CompanyFinancials2021[i].loc['Research Development','2021-06-30'].values[0] +
                                                      CompanyFinancials2021[i].loc['Research Development','2021-03-31'].values[0] +
                                                      CompanyFinancials2021[i].loc['Research Development','2020-12-31'].values[0] ) 
        else:
            reader1['researchDevelopment2021*'][i] = 0
# Research Development of 2017-2020
    if 'Research Development' in CompanyFinancials2017_2020[i].index:
        if pd.to_datetime('2020-12-31') in CompanyFinancials2017_2020[i].keys():
            reader1['researchDevelopment2020'][i] = CompanyFinancials2017_2020[i].loc['Research Development','2020-12-31'].values[0]
        if pd.to_datetime('2019-12-31') in CompanyFinancials2017_2020[i].keys():
            reader1['researchDevelopment2019'][i] = CompanyFinancials2017_2020[i].loc['Research Development','2019-12-31'].values[0]
        if pd.to_datetime('2018-12-31') in CompanyFinancials2017_2020[i].keys():
            reader1['researchDevelopment2018'][i] = CompanyFinancials2017_2020[i].loc['Research Development','2018-12-31'].values[0]
        if pd.to_datetime('2017-12-31') in CompanyFinancials2017_2020[i].keys():
            reader1['researchDevelopment2017'][i] = CompanyFinancials2017_2020[i].loc['Research Development','2017-12-31'].values[0]

In [10]:
pd.set_option('display.max_columns', 500)
reader1

,ticker,currency,financialCurrency,fullExchangeName,longName,shortName,exchangeTimezoneName,marketCap,price,currencyRatesToUSD,marketCap_USD,price_USD,fullTimeEmployees,city,country,totalRevenue2021*,totalRevenue2020,totalRevenue2019,totalRevenue2018,totalRevenue2017,netIncome2021*,netIncome2020,netIncome2019,netIncome2018,netIncome2017,researchDevelopment2021*,researchDevelopment2020,researchDevelopment2019,researchDevelopment2018,researchDevelopment2017,sharesOutstanding
0,688333.SS,CNY,CNY,Shanghai,"Xi'an Bright Laser Technologies Co.,Ltd.",XI'AN BRIGHT LASER TECHNOLOGIES,Asia/Shanghai,15626634240,193.42,6.37647,2.45067e+09,30.3334,733,Xiâan,China,5.18942e+08,4.12168e+08,3.21743e+08,2.91479e+08,2.19948e+08,-2.02202e+07,4.12168e+08,3.21743e+08,2.91479e+08,2.19948e+08,9.92458e+07,6.82305e+07,4.20343e+07,2.561e+07,2.32525e+07,80000000
1,DDD,USD,USD,NYSE,3D Systems Corporation,3D Systems Corporation,America/New_York,2608799232,20.31,1,2.6088e+09,20.31,1995,Rock Hill,United States,6.40097e+08,5.5724e+08,6.36354e+08,6.91545e+08,6.46069e+08,3.08422e+08,5.5724e+08,6.36354e+08,6.91545e+08,6.46069e+08,6.9023e+07,7.4143e+07,8.329e+07,9.5298e+07,9.4627e+07,128449000
2,DM,USD,USD,NYSE,"Desktop Metal, Inc.","Desktop Metal, Inc.",America/New_York,1464876032,4.71,1,1.46488e+09,4.71,202,Burlington,United States,6.4097e+07,1.647e+07,2.6439e+07,1.034e+06,,-1.38155e+08,1.647e+07,2.6439e+07,1.034e+06,,5.7594e+07,4.3136e+07,5.4656e+07,5.9607e+07,,311014016
3,SSYS,USD,USD,NasdaqGS,Stratasys Ltd.,"Stratasys, Ltd.",America/New_York,1486755456,22.71,1,1.48676e+09,22.71,1981,Eden Prairie,United States,5.82598e+08,5.20817e+08,6.3608e+08,6.63237e+08,6.68362e+08,-4.6116e+07,5.20817e+08,6.3608e+08,6.63237e+08,6.68362e+08,8.4636e+07,8.4012e+07,9.4253e+07,9.8964e+07,9.6237e+07,65467000
4,VJET,USD,EUR,NasdaqCM,voxeljet AG,voxeljet AG,America/New_York,43214268,6.15,1,4.32143e+07,6.15,251,Friedberg,Germany,2.28e+07,2.1567e+07,2.4602e+07,2.6009e+07,2.3178e+07,-1.5211e+07,2.1567e+07,2.4602e+07,2.6009e+07,2.3178e+07,6.535e+06,6.5e+06,7.183e+06,6.334e+06,5.528e+06,7026710
5,NNDM,USD,USD,NasdaqCM,Nano Dimension Ltd.,Nano Dimension Ltd.,America/New_York,995177344,3.88,1,9.95177e+08,3.88,92,Ness Ziona,Israel,4.933e+06,3.399e+06,7.07e+06,5.1e+06,829000,-5.8592e+07,3.399e+06,7.07e+06,5.1e+06,829000,3.0312e+07,9.878e+06,8.082e+06,8.623e+06,1.0819e+07,256488992
6,ONVO,USD,USD,NasdaqCM,"Organovo Holdings, Inc.","Organovo Holdings, Inc.",America/New_York,31426674,3.61,1,3.14267e+07,3.61,11,Solana Beach,United States,0,,,,,-1.1144e+07,,,,,0,,,,,8705450
7,SHPW,USD,USD,NYSE,"Shapeways Holdings, Inc.","Shapeways Holdings, Inc.",America/New_York,151168048,3.13,1,1.51168e+08,3.13,,Long Island City,United States,,3.1775e+07,3.3511e+07,,,0,3.1775e+07,3.3511e+07,,,0,5.592e+06,5.246e+06,,,48296500
8,MKFG,USD,USD,NYSE,Markforged Holding Corporation,Markforged Holding Corporation,America/New_York,1048199680,5.64,1,1.0482e+09,5.64,350,Watertown,United States,,7.1851e+07,7.2549e+07,,,0,7.1851e+07,7.2549e+07,,,0,1.7176e+07,2.027e+07,,,185851008
9,AM3D.DE,EUR,EUR,XETRA,SLM Solutions Group AG,SLM SOLUTIONS GRP AG,Europe/Berlin,372781216,17.56,0.887075,4.20236e+08,19.7954,429,LÃ¼beck,Germany,7.2639e+07,6.6789e+07,5.233e+07,7.5014e+07,8.5625e+07,-2.4739e+07,6.6789e+07,5.233e+07,7.5014e+07,8.5625e+07,0,None,None,None,None,21229000


In [11]:
reader1 = reader1.replace(np.nan, '', regex = True)

In [12]:
reader1 = reader1.replace(0, '', regex = True)

In [13]:
reader1 = reader1.replace(r'^\s*$', np.nan, regex=True)

In [14]:
reader1 = reader1.replace('Xiâan', "Xi'an", regex=True)

In [15]:
reader1 = reader1.replace('LÃ¼beck', "Lübeck", regex=True)

In [16]:
reader1

,ticker,currency,financialCurrency,fullExchangeName,longName,shortName,exchangeTimezoneName,marketCap,price,currencyRatesToUSD,marketCap_USD,price_USD,fullTimeEmployees,city,country,totalRevenue2021*,totalRevenue2020,totalRevenue2019,totalRevenue2018,totalRevenue2017,netIncome2021*,netIncome2020,netIncome2019,netIncome2018,netIncome2017,researchDevelopment2021*,researchDevelopment2020,researchDevelopment2019,researchDevelopment2018,researchDevelopment2017,sharesOutstanding
0,688333.SS,CNY,CNY,Shanghai,"Xi'an Bright Laser Technologies Co.,Ltd.",XI'AN BRIGHT LASER TECHNOLOGIES,Asia/Shanghai,15626634240,193.42,6.376475,2.450670e+09,30.333375,733.0,Xi'an,China,5.189421e+08,4.121681e+08,3.217428e+08,2.914792e+08,2.199483e+08,-2.022017e+07,4.121681e+08,3.217428e+08,2.914792e+08,2.199483e+08,9.924579e+07,6.823048e+07,4.203426e+07,2.561001e+07,2.325250e+07,80000000
1,DDD,USD,USD,NYSE,3D Systems Corporation,3D Systems Corporation,America/New_York,2608799232,20.31,1.000000,2.608799e+09,20.310000,1995.0,Rock Hill,United States,6.400970e+08,5.572400e+08,6.363540e+08,6.915450e+08,6.460690e+08,3.084220e+08,5.572400e+08,6.363540e+08,6.915450e+08,6.460690e+08,6.902300e+07,7.414300e+07,8.329000e+07,9.529800e+07,9.462700e+07,128449000
2,DM,USD,USD,NYSE,"Desktop Metal, Inc.","Desktop Metal, Inc.",America/New_York,1464876032,4.71,1.000000,1.464876e+09,4.710000,202.0,Burlington,United States,6.409700e+07,1.647000e+07,2.643900e+07,1.034000e+06,NaN,-1.381550e+08,1.647000e+07,2.643900e+07,1.034000e+06,NaN,5.759400e+07,4.313600e+07,5.465600e+07,5.960700e+07,NaN,311014016
3,SSYS,USD,USD,NasdaqGS,Stratasys Ltd.,"Stratasys, Ltd.",America/New_York,1486755456,22.71,1.000000,1.486755e+09,22.710000,1981.0,Eden Prairie,United States,5.825980e+08,5.208170e+08,6.360800e+08,6.632370e+08,6.683620e+08,-4.611600e+07,5.208170e+08,6.360800e+08,6.632370e+08,6.683620e+08,8.463600e+07,8.401200e+07,9.425300e+07,9.896400e+07,9.623700e+07,65467000
4,VJET,USD,EUR,NasdaqCM,voxeljet AG,voxeljet AG,America/New_York,43214268,6.15,1.000000,4.321427e+07,6.150000,251.0,Friedberg,Germany,2.280000e+07,2.156700e+07,2.460200e+07,2.600900e+07,2.317800e+07,-1.521100e+07,2.156700e+07,2.460200e+07,2.600900e+07,2.317800e+07,6.535000e+06,6.500000e+06,7.183000e+06,6.334000e+06,5.528000e+06,7026710
5,NNDM,USD,USD,NasdaqCM,Nano Dimension Ltd.,Nano Dimension Ltd.,America/New_York,995177344,3.88,1.000000,9.951773e+08,3.880000,92.0,Ness Ziona,Israel,4.933000e+06,3.399000e+06,7.070000e+06,5.100000e+06,8.290000e+05,-5.859200e+07,3.399000e+06,7.070000e+06,5.100000e+06,8.290000e+05,3.031200e+07,9.878000e+06,8.082000e+06,8.623000e+06,1.081900e+07,256488992
6,ONVO,USD,USD,NasdaqCM,"Organovo Holdings, Inc.","Organovo Holdings, Inc.",America/New_York,31426674,3.61,1.000000,3.142667e+07,3.610000,11.0,Solana Beach,United States,NaN,NaN,NaN,NaN,NaN,-1.114400e+07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8705450
7,SHPW,USD,USD,NYSE,"Shapeways Holdings, Inc.","Shapeways Holdings, Inc.",America/New_York,151168048,3.13,1.000000,1.511680e+08,3.130000,NaN,Long Island City,United States,NaN,3.177500e+07,3.351100e+07,NaN,NaN,NaN,3.177500e+07,3.351100e+07,NaN,NaN,NaN,5.592000e+06,5.246000e+06,NaN,NaN,48296500
8,MKFG,USD,USD,NYSE,Markforged Holding Corporation,Markforged Holding Corporation,America/New_York,1048199680,5.64,1.000000,1.048200e+09,5.640000,350.0,Watertown,United States,NaN,7.185100e+07,7.254900e+07,NaN,NaN,NaN,7.185100e+07,7.254900e+07,NaN,NaN,NaN,1.717600e+07,2.027000e+07,NaN,NaN,185851008
9,AM3D.DE,EUR,EUR,XETRA,SLM Solutions Group AG,SLM SOLUTIONS GRP AG,Europe/Berlin,372781216,17.56,0.887075,4.202363e+08,19.795388,429.0,Lübeck,Germany,7.263900e+07,6.678900e+07,5.233000e+07,7.501400e+07,8.562500e+07,-2.473900e+07,6.678900e+07,5.233000e+07,7.501400e+07,8.562500e+07,NaN,NaN,NaN,NaN,NaN,21229000


In [17]:
reader1.to_csv('/Users/songyang/Desktop/XUE/Data analysis projects/3DPrintingStockMarket/3DPrintingMarketCap_USD.csv')      

In [27]:
reader2 = pd.read_csv('/Users/songyang/Desktop/XUE/Data analysis projects/3DPrintingStockMarket/get3DPrintingStockData.csv')
reader2['Open_USD'] = ""
reader2['High_USD'] = ""
reader2['Low_USD'] = ""
reader2['Close_USD'] = ""
reader2['Adj Close_USD'] = ""
reader2['MarketCapOpen_USD'] = ""
reader2['MarketCapHigh_USD'] = ""
reader2['MarketCapLow_USD'] = ""
reader2['MarketCapClose_USD'] = ""
reader2['MarketCapAdj Close_USD'] = ""
#reader2['Volume_USD'] = "" #what the meaning of this "volume?" Is it Dollar volume?
for i in range (len(reader2['Ticker'])):
    for j in range (len(reader1['ticker'])):
        if reader2['Ticker'][i] == reader1['ticker'][j]:
            reader2['Open_USD'][i] = reader2['Open'][i]/reader1['currencyRatesToUSD'][j]
            reader2['High_USD'][i] = reader2['High'][i]/reader1['currencyRatesToUSD'][j]
            reader2['Low_USD'][i] = reader2['Low'][i]/reader1['currencyRatesToUSD'][j]
            reader2['Close_USD'][i] = reader2['Close'][i]/reader1['currencyRatesToUSD'][j]
            reader2['Adj Close_USD'][i] = reader2['Adj Close'][i]/reader1['currencyRatesToUSD'][j]
            reader2['MarketCapOpen_USD'][i] = reader2['Open'][i]/reader1['currencyRatesToUSD'][j]*float(reader1['sharesOutstanding'][j])
            reader2['MarketCapHigh_USD'][i] = reader2['High'][i]/reader1['currencyRatesToUSD'][j]*float(reader1['sharesOutstanding'][j])
            reader2['MarketCapLow_USD'][i] = reader2['Low'][i]/reader1['currencyRatesToUSD'][j]*float(reader1['sharesOutstanding'][j])
            reader2['MarketCapClose_USD'][i] = reader2['Close'][i]/reader1['currencyRatesToUSD'][j]*float(reader1['sharesOutstanding'][j])
            reader2['MarketCapAdj Close_USD'][i] = reader2['Adj Close'][i]/reader1['currencyRatesToUSD'][j]*float(reader1['sharesOutstanding'][j])
#            reader2['Volume_USD'][i] = reader2['Volume'][i]/reader1.currencyRatesToUSD[j] 
reader2.to_csv('/Users/songyang/Desktop/XUE/Data analysis projects/3DPrintingStockMarket/get3DPrintingStockData_USD.csv')      

In [28]:
pd.set_option('display.max_rows', 2000)
reader2.head(500)

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,Open_USD,High_USD,Low_USD,Close_USD,Adj Close_USD,MarketCapOpen_USD,MarketCapHigh_USD,MarketCapLow_USD,MarketCapClose_USD,MarketCapAdj Close_USD
0,2019-07-22,61.000000,96.000000,43.400002,64.330002,64.186386,14826307,688333.SS,9.56641,15.0553,6.80627,10.0886,10.0661,7.65313e+08,1.20443e+09,5.44502e+08,8.07092e+08,8.0529e+08
1,2019-07-23,54.500000,62.000000,54.000000,57.400002,57.271854,5875325,688333.SS,8.54704,9.72324,8.46863,9.00184,8.98174,6.83763e+08,7.77859e+08,6.7749e+08,7.20147e+08,7.18539e+08
2,2019-07-24,61.209999,68.000000,61.180000,67.989998,67.838211,9373542,688333.SS,9.59935,10.6642,9.59464,10.6626,10.6388,7.67948e+08,8.53136e+08,7.67571e+08,8.5301e+08,8.51106e+08
3,2019-07-25,65.459999,76.000000,65.029999,71.019997,70.861443,9534023,688333.SS,10.2659,11.9188,10.1984,11.1378,11.1129,8.21269e+08,9.53505e+08,8.15874e+08,8.91025e+08,8.89036e+08
4,2019-07-26,68.000000,70.790001,66.180000,67.339996,67.189659,5514648,688333.SS,10.6642,11.1017,10.3788,10.5607,10.5371,8.53136e+08,8.8814e+08,8.30302e+08,8.44855e+08,8.42969e+08
5,2019-07-29,70.019997,77.000000,70.019997,76.599998,76.428986,7747681,688333.SS,10.981,12.0756,10.981,12.0129,11.9861,8.78479e+08,9.66051e+08,8.78479e+08,9.61033e+08,9.58887e+08
6,2019-07-30,79.570000,85.699997,77.349998,78.290001,78.115219,9502271,688333.SS,12.4787,13.44,12.1305,12.2779,12.2505,9.98295e+08,1.0752e+09,9.70442e+08,9.82236e+08,9.80043e+08
7,2019-07-31,77.680000,84.879997,75.519997,83.900002,83.712692,8517093,688333.SS,12.1823,13.3114,11.8435,13.1577,13.1284,9.74582e+08,1.06491e+09,9.47483e+08,1.05262e+09,1.05027e+09
8,2019-08-01,82.000000,98.500000,81.110001,94.879997,94.668175,9810842,688333.SS,12.8598,15.4474,12.7202,14.8797,14.8465,1.02878e+09,1.23579e+09,1.01762e+09,1.19038e+09,1.18772e+09
9,2019-08-02,91.000000,106.220001,90.010002,98.000000,97.781212,10518009,688333.SS,14.2712,16.6581,14.116,15.369,15.3347,1.1417e+09,1.33265e+09,1.12928e+09,1.22952e+09,1.22677e+09


In [20]:
# yf.Ticker('688333.SS').(financials, quarterly_financials, major_holders, cashflow, 
        # quarterly_cashflow, recommendations, history(period="1mo"), analysis)

In [1]:
yf.Ticker('688333.SS').info.keys

NameError: name 'yf' is not defined

Clean up downloaded data. 
Today's data isn't always correct, so code below will remove today's data.
Sometimes, duplicate data are downloaded, code below will remove duplicates as well.

In [21]:
## Clean up downloaded data. 
#if len(df_stocks)>0: 
#    df_stocks=df_stocks.reset_index()
#    df_stocks=df_stocks[df_stocks['Date']<pd.to_datetime('today').floor('d')]
#if len(df_stocks)>0:
#    df_stocks['Total']=df_stocks['High']+df_stocks['Volume']
#    df_stocks=df_stocks[df_stocks['Total'] == df_stocks.groupby(['name', 'Date'])['Total'].transform('max')]
#    df_stocks=df_stocks.drop('Total', axis=1)
#    df_stocks=df_stocks.drop_duplicates()
#    df_stocks=df_stocks.set_index('Date')
#    stocks=stocks.set_index('Date')
#    stocks=stocks.append(df_stocks, sort=False, ignore_index=False)      
#    stocks.to_excel('/Users/songyang/Desktop/XUE/Data analysis projects/3DPrintingStockMarket/covidStocks.xlsx', sheet_name='covidStocks')

In [22]:
## Code below check if any tickers were missing from downloaded data. 
#if len(df_stocks)<len(tickers):
#    missing_ticker=set(tickers.keys())-set(df_stocks['name'])
#    print('Missing:', missing_ticker)
#else:
#    print('All Tickers download')

In [23]:
### Code use for adding new ticker to the dataset
# ticker = 'psxp'
# name='Phillips66'  
# df_new=yf.download(ticker )
# df_new['name']=name
# df_new=df_new.reset_index()
# stocks=pd.read_excel('google_drive_path/file_name.xlsx', sheet_name='Sheet1')
# new_ticker=stocks.append(df_new)
# new_ticker=new_ticker.set_index('Date')
# with pd.ExcelWriter('google_drive_path/file_name.xlsx'',engine='openpyxl') as writer:
#     new_ticker.to_excel(writer)